In [ ]:
from PIL import Image
import datetime
import re

import torchvision.models as models
from torchvision.datasets import ImageFolder
import torch
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from collections import OrderedDict
import pandas as pd
import hiddenlayer as hl
import glob
from PIL import Image
import cv2
import random
from gradcam import *
from fnmatch import fnmatch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

save_path = "../../DATASETS/ISSI/ORGANIZED/"


In [ ]:

years = ["2007","2008","2009","2010","2011","2012","2013","2014","2015","2016"]

def look_for_file(rootdir, name):
    regex = re.compile(name+"*.jpeg")

    for root, dirs, files in os.walk(rootdir):
        for file in files:
            #print("file : "+str(file))
            if name in file:
                #print("file : "+str(file))
                return file
count = {
            '0':0,
            '1':0,
            '2':0,
            '3':0,
            '4':0,
            '5':0,
        }
for year in years:
    df = pd.read_csv("../../DATASETS/ISSI/2007-2016 Organisation Summary/"+year+"-Summary-"+year+"-Summary.csv")
    print(" File : "+str("../../DATASETS/ISSI/2007-2016 Organisation Summary/"+year+"-Summary-"+year+"-Summary.csv"))
    #print(df)
    for index, row in df.iterrows():
        cols = list(df.columns)
        classes = {
            '0':0,
            '1':0,
            '2':0,
            '3':0,
            '4':0,
            '5':0,
        }
        try:
            classes[ str(  row[cols[2]]  )  ] += 1
        except:
            classes['0'] += 1
        try:
            classes[ str(  row[cols[3]]  )  ] += 1
        except:
            classes['0'] += 1
        try:
            classes[ str(  row[cols[4]]  )  ] += 1
        except:
            classes['0'] += 1
        try:
            classes[ str(  row[cols[5]]  )  ] += 1
        except:
            classes['0'] += 1
            
        try:
            classes[ str(  row[cols[6]]  )  ] += 1
        except:
            classes['0'] += 1
            
        try:
            classes[ str(  row[cols[7]]  )  ] += 1
        except:
            classes['0'] += 1
        
        #print("row : "+str(row))
        datetime_object = datetime.datetime.strptime(row['Day'].split()[0], "%b")
        month_number = datetime_object.month
        day = row['Day'].split()[1]
        path = "../../DATASETS/ISSI/images/"
        name_file = "Aqua_CorrectedReflectance{year:04d}{month:02d}{day:02d}" #".png"
        ex_path = name_file.format(year=int(year), month=int(month_number), day=int(day))
        best = max(classes, key=classes.get)
        filename = look_for_file(path,ex_path)
        file_img = path+filename
        img = Image.open(file_img)
        imges = cv2.imread(file_img,1)
        """
        if [0, 0, 0] in imges:
            continue
        """
        if best in ['1','2','3','4','5']:
            sav_path = save_path+str(best)+"/"+str(count[best])+".jpeg"
            #print("save path : "+str(sav_path))
            count[best] += 1
            img.save(sav_path)
        #print("Best : "+str(best))
        #print("Classes : "+str(classes))
        

In [ ]:
PERCENTAGE_TEST = 0.4
BATCH_SIZE = 32
normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )
transformer = transforms.Compose([
    transforms.Resize([768, 512]),
    # you can add other transformations in this list
    transforms.ToTensor(),
    normalize,
])


dataset = ImageFolder(root="../../DATASETS/ISSI/ORGANIZED/", transform=transformer)

dataloader  = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1) 


In [ ]:
device = 'cpu' 
"""
model = models.resnet18(pretrained=True)


model.fc = nn.Sequential(
    nn.Linear(512, 100),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(100, 20),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(20, 4),
    nn.LogSoftmax(dim=1)
    )
model.load_state_dict(torch.load("../zooniverse-resized-image_state_dict", map_location=device))

"""
model = torch.load("../zooniverse-resized-image_net_model", map_location=device)
"""
transforms = [ hl.transforms.Prune('Constant') ] # Removes Constant nodes from graph.

graph = hl.build_graph(model, torch.zeros([1, 3, 224, 224]))
graph.theme = hl.graph.THEMES['blue'].copy()
graph.save('model_hiddenlayer', format='png')
"""


In [ ]:
nb_classes = 4

confusion_matrix = torch.zeros(nb_classes, nb_classes)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1


In [ ]:


cfs_mat = confusion_matrix
associations = ['Fish', 'Flower', 'Gravel', 'Sugar']
associations_model = ['Flower', 'Fish', 'Gravel', 'Sugar']
print(confusion_matrix[0])
fig, ax = plt.subplots(figsize=(20,10)) 
plt.title("Confusion Matrix for ISSI Images")
sns.set(font_scale=1.8)
sns.heatmap(cfs_mat, annot=True, fmt='', ax=ax, linewidths=.9, yticklabels=associations, xticklabels=associations_model)
"""
fig, ax = plt.subplots(figsize=(20,10))
print(cfs_mat[:4,:].shape)
print(   np.sum(  cfs_mat[:4,:], axis=1  ).shape  )
print(   (  cfs_mat[:4,:].T/np.sum(cfs_mat[:4,:], axis=1)    ).shape  )
plt.title("Confusion Matrix Clusters MODIS model %")
sns.heatmap( (cfs_mat[:4,:].T/np.sum(cfs_mat[:4,:], axis=1)).T, annot=True, 
            fmt='.2%', cmap='Blues' , yticklabels=associations)
"""

In [ ]:
print(dataset.classes)

In [ ]:
from PIL import Image
import datetime
import re

import torchvision.models as models
from torchvision.datasets import ImageFolder
import torch
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from collections import OrderedDict
import pandas as pd
import hiddenlayer as hl
import glob
from PIL import Image
import cv2
import random
from gradcam import *
from fnmatch import fnmatch
save_path = "../../DATASETS/ISSI/ORGANIZED/"

device = 'cpu' 

model = models.resnet18(pretrained=True)


model.fc = nn.Sequential(
    nn.Linear(512, 100),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(100, 20),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(20, 4),
    nn.LogSoftmax(dim=1)
    )
model.load_state_dict(torch.load("../zooniverse-resized-image_state_dict", map_location=device))


root = save_path
pattern = "*.jpeg"
pathes = []
for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            pathes.append(os.path.join(path, name))
#pathes =  ["../data/3.jpg"]
#pathes = pathes.reverse()
associations = ["Fish", "Flower", "Gravel", "Sugar"]
done = ["Fish", "Flower", "Gravel", "Sugar"]
count = 0
while len(done)!=0:
    #print("Image "+str(count))
    """
    inputs = inputs.to(device)
    classes = classes.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    print("SHAPE : "+str(inputs.shape))
    """
    path = random.choice(pathes)
    pil_img = Image.open(path)
    inputs = transformer(pil_img).unsqueeze_(0)
    inputs = inputs.to(device)
    outputs = model(inputs)
    
    _, preds = torch.max(outputs, 1)
    """
    print("path : "+str(path))
    print("outputs : "+str(outputs))
    print("ASSO : "+str(associations[int(preds[0])]))
    print("preds : "+str(int(preds[0])))
    """
    if len(done) == 0:
        break
    if associations[preds[0]] not in done:
        continue
    else:
        done.remove(associations[int(preds[0])])
        predict_image(model, path, str(associations[preds[0]]))


In [ ]:
from PIL import Image
import datetime
import re

import torchvision.models as models
from torchvision.datasets import ImageFolder
import torch
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from collections import OrderedDict
import pandas as pd
import hiddenlayer as hl
import glob
from PIL import Image
import cv2
import random
from gradcam import *
from fnmatch import fnmatch
save_path = "../../DATASETS/CLASSIF_RESIZED/"

device = 'cpu' 

model = models.resnet18(pretrained=True)


model.fc = nn.Sequential(
    nn.Linear(512, 100),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(100, 20),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(20, 4),
    nn.LogSoftmax(dim=1)
    )
model.load_state_dict(torch.load("../zooniverse-resized-image_state_dict", map_location=device))


root = save_path
pattern = "*.jpeg"
pathes = []
for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            pathes.append(os.path.join(path, name))
#pathes =  ["../data/3.jpg"]
#pathes = pathes.reverse()
associations = ["Fish", "Flower", "Gravel", "Sugar"]
done = ["Fish", "Flower", "Gravel", "Sugar"]
count = 0
while len(done)!=0:
    #print("Image "+str(count))
    """
    inputs = inputs.to(device)
    classes = classes.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    print("SHAPE : "+str(inputs.shape))
    """
    path = random.choice(pathes)
    pil_img = Image.open(path)
    inputs = transformer(pil_img).unsqueeze_(0)
    inputs = inputs.to(device)
    outputs = model(inputs)
    
    _, preds = torch.max(outputs, 1)
    """
    print("path : "+str(path))
    print("outputs : "+str(outputs))
    print("ASSO : "+str(associations[int(preds[0])]))
    print("preds : "+str(int(preds[0])))
    """
    if len(done) == 0:
        break
    if associations[preds[0]] not in done:
        continue
    else:
        done.remove(associations[int(preds[0])])
        predict_image(model, path, str(associations[preds[0]]))


In [ ]:
from PIL import Image
import datetime
import re

import torchvision.models as models
from torchvision.datasets import ImageFolder
import torch
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from collections import OrderedDict
import pandas as pd
import hiddenlayer as hl
import glob
from PIL import Image
import cv2
import random
from gradcam import *
from fnmatch import fnmatch
save_path = "../../DATASETS/EURE4CA/tmp/train/"

device = 'cpu' 

model = models.resnet18(pretrained=True)


model.fc = nn.Sequential(
    nn.Linear(512, 100),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(100, 20),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(20, 4),
    nn.LogSoftmax(dim=1)
    )
model.load_state_dict(torch.load("../zooniverse-resized-image_state_dict", map_location=device))


root = save_path
pattern = "*.png"
pathes = []
for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            pathes.append(os.path.join(path, name))
#pathes =  ["../data/3.jpg"]
#pathes = pathes.reverse()
associations = ["Fish", "Flower", "Gravel", "Sugar"]
done = ["Fish", "Flower", "Gravel", "Sugar"]
count = 0
while len(done)!=0:
    #print("Image "+str(count))
    """
    inputs = inputs.to(device)
    classes = classes.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    print("SHAPE : "+str(inputs.shape))
    """
    path = random.choice(pathes)
    pil_img = Image.open(path)
    inputs = transformer(pil_img).unsqueeze_(0)
    inputs = inputs.to(device)
    outputs = model(inputs)
    
    _, preds = torch.max(outputs, 1)
    """
    print("path : "+str(path))
    print("outputs : "+str(outputs))
    print("ASSO : "+str(associations[int(preds[0])]))
    print("preds : "+str(int(preds[0])))
    """
    if len(done) == 0:
        break
    if associations[preds[0]] not in done:
        continue
    else:
        done.remove(associations[int(preds[0])])
        predict_image(model, path, str(associations[preds[0]]))


In [ ]:
from PIL import Image
import datetime
import re

import torchvision.models as models
from torchvision.datasets import ImageFolder
import torch
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from collections import OrderedDict
import pandas as pd
import hiddenlayer as hl
import glob
from PIL import Image
import cv2
import random
from gradcam import *
from fnmatch import fnmatch
IMAGE_FOLDER = "../../DATASETS/CLASSIF_RESIZED/"
BATCH_SIZE = 16
device = 'cpu' 

model = models.resnet18(pretrained=True)


model.fc = nn.Sequential(
    nn.Linear(512, 100),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(100, 20),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(20, 4),
    nn.LogSoftmax(dim=1)
    )
model.load_state_dict(torch.load("../zooniverse-resized-image_state_dict", map_location=device))

normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )
transformer = transforms.Compose([
    transforms.Resize([768, 512]),
    # you can add other transformations in this list
    transforms.ToTensor(),
    normalize,
])
dataset = ImageFolder(root=IMAGE_FOLDER, transform=transformer)
print(len(dataset))
n = len(dataset)
n_test = int(0.1 * n)  # take ~10% for test
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-n_test, n_test], generator=torch.Generator().manual_seed(42))


train_dataloader  = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=1) 
test_dataloader  = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)




In [ ]:

nb_classes = 4

confusion_matrix = torch.zeros(nb_classes, nb_classes)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_dataloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1


In [ ]:

cfs_mat = confusion_matrix.numpy()
associations = ['Fish', 'Flower', 'Gravel', 'Sugar']
print(confusion_matrix[0])
fig, ax = plt.subplots(figsize=(20,10)) 
plt.title("Confusion Matrix on test Zooniverse resized Images")
sns.set(font_scale=1.8)
sns.heatmap(cfs_mat, annot=True, fmt='', ax=ax, linewidths=.9, yticklabels=associations, xticklabels=associations)

fig, ax = plt.subplots(figsize=(20,10))
plt.title("Confusion Matrix on test Zooniverse resized Images %")
a = cfs_mat.T
b = np.sum(cfs_mat, axis=1)
sns.heatmap( (cfs_mat.T/np.sum(cfs_mat, axis=1)).T, annot=True, 
            fmt='.2%', cmap='Blues' , yticklabels=associations, xticklabels=associations)


In [ ]:
print("N = "+str(n))